# Einleitung: Modul `graphfw.params.sql_connection_check`

Das Modul **`graphfw.params.sql_connection_check`** unterstützt dich beim **Einrichten, Prüfen und Persistieren** von SQL‑Verbindungsparametern in deiner `config.json`. Es liest bestehende Einstellungen (inkl. optionaler ENV‑Overrides), führt eine **Mehrwege‑Diagnose** über `graphfw.core.odbc_utils` durch, leitet daraus einen **funktionsfähigen Konfigurationsvorschlag** ab und kann diesen optional **atomar in die `config.json` schreiben** (mit Backup und sicherem Passwort‑Handling).

---

## Wann einsetzen?

* **Erstkonfiguration** einer SQL‑Verbindung: korrekte Treiber/Flags finden.
* **Fehleranalyse**: warum schlägt eine Verbindung fehl, welche Versuche wurden gemacht?
* **Konfigurationspflege**: einen getesteten Kandidaten direkt in `config.json` übernehmen – ohne Passwörter zu loggen.

---

## Erwartetes JSON‑Schema pro Node

Das Modul ist auf folgendes Schema (unter `sql.<node>`) ausgelegt:

```json
{
  "server": "myserver.domain.tld",
  "db_name": "BI_RAW",
  "username": "svc_user",
  "password": "CHANGE_ME",
  "driver": "ODBC Driver 18 for SQL Server",
  "auth": "sql" | "trusted" | "aad-password" | "aad-integrated" | "aad-interactive" | "aad-msi" | "aad-sp",
  "params": { "Encrypt": "yes", "TrustServerCertificate": "no" }
}
```

> `params` ist ein **Dict** (keine String‑Konkatenation). DSN‑basierte Einträge sind ebenfalls möglich (Feld `dsn`).

---

## So arbeitet das Modul (Workflow)

1. **Settings laden**: Liest `sql.<node>` aus `config.json` (bzw. ENV‑Overrides wie `SQL_SERVER`, `SQL_PARAMS_JSON`).
2. **Diagnose ausführen**: Übergibt die Settings an `core.odbc_utils.diagnose_with_fallbacks` (SQLAlchemy+pyodbc, pyodbc‑direct, ADO/OLE DB, optional pymssql).
3. **Kandidaten bauen**: Nimmt den **ersten erfolgreichen Attempt** (falls vorhanden) + deine Basis‑Settings und erzeugt einen **geordneten** Konfigurationsvorschlag (DSN bevorzugt, sonst explizit). Passwörter erscheinen **nur** als Platzhalter.
4. **Optionale Persistenz**: Auf Wunsch wird der Kandidat **atomar** in die `config.json` geschrieben (mit **Backup**, `keep_existing_password`, `dry_run`).

---

## Funktionen im Überblick

### `connect_and_check(node, *, show_drivers=True, show_dsns=True, return_config_json=True, write_config=False, config_path=CONFIG_PATH, keep_existing_password=True, dry_run=False) -> (ok, diag, config_json, write_info)`

**Zweck:** End‑to‑End: lädt Settings, listet (optional) ODBC‑Treiber/DSNs, führt die Diagnose aus, erzeugt den JSON‑Vorschlag und **optional** schreibt ihn in die `config.json`.

* **Parameter**

  * `node`: Pfad unter `sql` (z. B. `"connections.sql_basic"`).
  * `show_drivers`/`show_dsns`: Ausgabe installierter Treiber/DSNs.
  * `return_config_json`: JSON‑Vorschlag als String zurückgeben.
  * `write_config`: Vorschlag direkt in Datei übernehmen.
  * `config_path`: Pfad zur `config.json`.
  * `keep_existing_password`: vorhandenes Passwort **nicht überschreiben**, wenn nur Platzhalter geliefert wird.
  * `dry_run`: Schreiben simulieren.
* **Rückgabe**

  * `ok: bool`: Mindestens ein Verbindungsweg erfolgreich?
  * `diag: dict`: Diagnose mit `summary`, `attempts[]`, `suggestions[]`.
  * `config_json: Optional[str]`: Einfügbarer Block `{ "sql": { node: ... } }`.
  * `write_info: Optional[dict]`: Infos zum Schreibvorgang (Pfad, Backup, maskiertes Ergebnis, `written`).

---

### `build_config_candidate(settings, diag) -> dict`

**Zweck:** Erzeugt aus **Basis‑Settings** + **erstem erfolgreichen Attempt** einen **geordneten** Konfigurationskandidaten.

* **Regeln**

  * **DSN** wird bevorzugt, andernfalls explizite Felder (`driver`, `server`, `db_name`, `username`).
  * `auth` wird übernommen bzw. heuristisch bestimmt (`trusted` bei `Trusted_Connection`, sonst `sql` bei Benutzer+Passwort).
  * `params` bleibt ein **Dict**.
  * **Passwort**: nur als Platzhalter `<<<SET_SECRET_HERE>>>`, sofern für den Auth‑Modus nötig (`sql`, `aad-password`, `aad-sp`).
  * **Deterministische Reihenfolge** der Schlüssel.

---

### `render_config_json(node, entry) -> str`

**Zweck:** Verpackt den Kandidaten in den standardisierten Block zur direkten Übernahme in die `config.json`:

```json
{
  "sql": {
    "<node>": { ... }
  }
}
```

---

### `apply_config_update(*, config_path, node, new_entry, create_backup=True, keep_existing_password=True, dry_run=False) -> dict`

**Zweck:** Schreibt/merged den **bereits validierten** Kandidaten unter `sql.<node>` in die echte `config.json`.

* **Sicherheit & Robustheit**

  * **Atomic write** via Temp‑Datei + `os.replace`.
  * **Backup**: `<config>.bak-YYYYmmddHHMMSS`.
  * **Secret‑Schutz**: vorhandenes Passwort wird **beibehalten**, wenn `keep_existing_password=True` und `new_entry.password` fehlt/Platzhalter ist.
  * **Validierung**: DSN **oder** (`driver`, `server`, `db_name`). Auth‑spezifische Checks (z. B. kein leerer Passwort‑String).
* **Rückgabe**: Metadaten inkl. maskierter Vorher/Nachher‑Einträge.

---

### `quick_check(engine) -> Any`

**Zweck:** Mini‑Smoke‑Test für vorhandene SQLAlchemy‑Engines (`SELECT 1`). Optional – unabhängig von der obigen Diagnosekette.

---

### `show_settings(settings) -> None`

**Zweck:** Maskierte Kurzansicht wichtiger Felder (ohne Secrets) – hilfreich beim Debugging.

---



## Beispiele

### 1) Nur prüfen & Vorschlag anzeigen (Datei bleibt unverändert)

In [ ]:
from pathlib import Path
import sys
sys.path.insert(0, "..")
from graphfw.params.sql_connection_check import connect_and_check, CONFIG_PATH

CONFIG_PATH = r"C:\\python\\Scripts\\config_sql.json"

ok, diag, cfg_json, write_info = connect_and_check(
    "connections.sql_basic",
    show_drivers=True,
    show_dsns=True,
    return_config_json=True,
    write_config=False,
    config_path=CONFIG_PATH,
)

### 2) Kandidat direkt persistieren (mit Backup, Secret‑Erhalt)

In [ ]:
from pathlib import Path
import sys
sys.path.insert(0, "..")
from graphfw.params.sql_connection_check import connect_and_check, CONFIG_PATH

CONFIG_PATH = r"C:\\python\\Scripts\\config_sql.json"

ok, diag, cfg_json, write_info = connect_and_check(
    "connections.sql_basic",
    return_config_json=True,
    write_config=True,                # schreibt in die Datei
    keep_existing_password=True,      # vorhandenes Secret bleibt erhalten
    dry_run=False,                    # zum Testen zuerst True setzen
    config_path=CONFIG_PATH,
)


### 3) Manuell schreiben (falls du den Kandidaten selbst definierst)

In [ ]:
from pathlib import Path
import sys
sys.path.insert(0, "..")
from graphfw.params.sql_connection_check import apply_config_update, CONFIG_PATH

CONFIG_PATH = r"C:\\python\\Scripts\\config_sql.json"

entry = {
    "driver": "ODBC Driver 18 for SQL Server",
    "server": "myserver.domain.tld,1433",
    "db_name": "BI_RAW",
    "username": "svc_user",
    "password": "<<<SET_SECRET_HERE>>>",
    "auth": "sql",
    "params": { "Encrypt": "yes", "TrustServerCertificate": "no" }
}

info = apply_config_update(
    config_path=CONFIG_PATH,
    node="connections.sql_basic",
    new_entry=entry,
    create_backup=True,
    keep_existing_password=True,
    dry_run=False,
)
print(info)

## Performance‑Tipps

* **Treiber/Flags fixieren** (`driver`, `params.Encrypt`, `params.TrustServerCertificate`, `server` inkl. Port), um Varianten‑Toggling zu vermeiden.
* **Zeitlimits** in `params` setzen (z. B. `Connect Timeout`, `LoginTimeout` = 3–5s), um Fehlschläge schneller abzubrechen.
* `show_drivers=False`, `show_dsns=False` spart kleine Mengen an Zeit, die Hauptwirkung kommt jedoch von präzisen Settings.

---

## Sicherheit & Grenzen

* Passwörter werden **nie** geloggt. Im Vorschlag steht ein **Platzhalter** – echte Secrets bitte sicher via ENV/Secret‑Store einbringen.
* Das Modul **ändert** die Datei nur, wenn `write_config=True` (oder `apply_config_update` explizit aufgerufen wird).
* Ohne installierte ODBC‑Treiber kann die Diagnose keine funktionsfähige Verbindung aufbauen.

---

## Abhängigkeiten

* `graphfw.core.odbc_utils` (Diagnosepfade, Treiber/DSN‑Listing)
* optional `sqlalchemy`, `pyodbc`, ggf. `pywin32` (ADO/OLE DB), `pymssql` (Fallback)

> Die Funktion `load_sql_settings` wird – wie im funktionierenden Altpfad – primär aus `graphfw.core.config` bezogen; ein interner Fallback‑Loader ist enthalten, falls diese Funktion in deiner Umgebung nicht verfügbar ist.


In [1]:
# Imports & Setup (Diagnose)
from pathlib import Path
import sys
sys.path.insert(0, "..")

from sqlalchemy import text
from graphfw.core.config import load_sql_settings, save_sql_settings, SQLSettings, __version__ as sql_config_version
from graphfw.core.odbc_utils import (
    list_odbc_drivers, list_odbc_data_sources, diagnose_with_fallbacks,
)

CONFIG_PATH = r"C:\python\Scripts\config_sql.json"

def connect_and_check(node: str, *, show_drivers: bool = True, show_dsns: bool = True):
    print(f"\n=== Node: {node} ===")
    if show_drivers:
        print("ODBC-Treiber (SQL Server):", list_odbc_drivers())
    if show_dsns:
        print("ODBC-DSNs:", list_odbc_data_sources())

    settings, info = load_sql_settings(config_path=CONFIG_PATH, node=node, env_override=True)
    print("Quelle:", info.get("source"), "| Node:", info.get("node_path"))
    print("Settings:", settings.as_dict(mask_secrets=True))

    ok, diag = diagnose_with_fallbacks(settings)
    print(diag["summary"])
    for i, att in enumerate(diag["attempts"], 1):
        status = "OK" if (att.get("ok") or ("error" not in att)) else "ERR"
        print(f"  [{i:02d}] {att.get('method','?'):<18} | driver={att.get('driver') or att.get('provider')} "
              f"| params={att.get('params') or ''} | {att.get('duration_s','-')}s")
        if att.get("error"):
            print("       ", att["error"])
    if diag.get("suggestions"):
        print("\nHinweise:")
        for s in diag["suggestions"]:
            print(" -", s)
    return ok, diag

# Aufruf:
connect_and_check("connections.sql_basic")



=== Node: connections.sql_basic ===
ODBC-Treiber (SQL Server): ['ODBC Driver 18 for SQL Server', 'ODBC Driver 17 for SQL Server', 'SQL Server']
ODBC-DSNs: {'MS Access Database': 'Microsoft Access Driver (*.mdb, *.accdb)', 'Excel Files': 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'dBASE Files': 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)'}
Quelle: json | Node: connections.sql_basic
Settings: {'server': 'srv-rz-bi-03', 'db_name': 'BI_RAW', 'username': 'sqlODATAImporter', 'password': '****', 'driver': 'ODBC Driver 18 for SQL Server', 'params': 'Encrypt=yes&TrustServerCertificate=yes'}
Verbindung OK (sqlalchemy+pyodbc)
  [01] sqlalchemy+pyodbc  | driver=ODBC Driver 18 for SQL Server | params=Encrypt&TrustServerCertificate | 7.033s
        OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]SSL Provider: Die Zertifikatkette wurde von einer nicht vertrauenswürdigen Zertifizierungsstelle ausgestellt.\r\n (-214689301

(True,
 {'summary': 'Verbindung OK (sqlalchemy+pyodbc)',
  'attempts': [{'method': 'sqlalchemy+pyodbc',
    'driver': 'ODBC Driver 18 for SQL Server',
    'params': 'Encrypt&TrustServerCertificate',
    'ok': False,
    'duration_s': 7.033,
    'error': "OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]SSL Provider: Die Zertifikatkette wurde von einer nicht vertrauenswürdigen Zertifizierungsstelle ausgestellt.\\r\\n (-2146893019) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Client unable to establish connection. For solutions related to encryption errors, see https://go.microsoft.com/fwlink/?linkid=2226722 (-2146893019)')\n(Background on this error at: https://sqlalche.me/e/20/e3q8) | orig=('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]SSL Provider: Die Zertifikatkette wurde von einer nicht vertrauenswürdigen Zertifizierungsstelle ausgestellt.\\r\\n (-2146893019) (SQLDriverConnect); [08001] [